In [1]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
model = model_from_json(open('fer.json','r').read())
model.load_weights('fer.h5')

In [3]:
face_haar_cascade = cv2.CascadeClassifier('C:/Users/vivek/Desktop/emotion_analysis/haar_cascade/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
while True:
    ret, test_img = cap.read()
    if not ret:
        continue
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5)
    
    for (x,y,w,h) in faces_detected:
        
        cv2.rectangle(test_img, (x,y), (x+w, y+h), (0,255,0), thickness=2)
        roi_gray = gray_img[y:y+w, x:x+h]
        roi_gray = cv2.resize(roi_gray, (48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255
        
        predictions = model.predict(img_pixels)
#         print(predictions)
        max_index = np.argmax(predictions[0])
        
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        
        predicted_emotion = emotions[max_index]
        
        cv2.putText(test_img, predicted_emotion, (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), thickness=1)
        
    resized_img = cv2.resize(test_img, (1000,700))
    cv2.imshow('facial emotion recognition', resized_img)
    
    if cv2.waitKey(10) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()